In [37]:
#import os
import math
import fiona 
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Polygon, Point
from explode import explode

In [38]:
#SET COUNTRY <----------------------------------------------------------------------------------CHANGE
#n = 'TUR' 
n = 'GRC'
#n = 'ESP'
#n = 'HRV' 
#n = 'ITA' 
#n = 'CYP' 
#n = 'MLT' 
#n = 'MLT' 
#n = 'ALB' 

#SET CRS #<-------------------------------------------------------------------------------------CHANGE
#crs = 'EPSG:5637' #turkey
crs = 'EPSG:2100' #greece
#crs = 'EPSG:2062' #spain
#crs = 'EPSG:3765' #croatia
#crs = 'EPSG:7794' #italy
#crs = 'EPSG:6312' #cyprus
#crs = 'EPSG:3034' #malta
#crs = 'EPSG:2154' #france
#crs = 'EPSG:6962' #albania
wgs84= {'init' :'EPSG:4326'}

#open file
f = '../targets_2_simrdwn/' + n + '/'
file = f + 'predictions_' + n + '.csv'
detections = pd.read_csv(file, delimiter=',', header=0)
print(len(detections), 'total SIMRDWN predictions')
#detections = detections[0:1000]
detections.head()

17834 total SIMRDWN predictions


,img_file,img_width,img_height,label,xmin,ymin,xmax,ymax
0,image_18_35_41838_24_96507_40_5061_22_66807_40...,2234,2175,c,1706,1195,1768,1335
1,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,c,1111,954,1133,978
2,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,c,1145,954,1168,979
3,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,c,1023,953,1048,982
4,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,c,1195,954,1217,978


In [39]:
x = detections['img_file'].iloc[1]
x.split('_')

['image',
 '18',
 '36',
 '56134',
 '26',
 '39112',
 '38',
 '94454',
 '20',
 '86329',
 '38',
 '95323',
 '20',
 '85137.jpeg']

In [40]:
#georeference prediction bounding boxes and convert to centroid
selected = detections

lon_list, lat_list, radius_list, diameter_list = [], [], [], []
for i in range(len(selected)):
    img = detections['img_file'].iloc[i]

#     #extract bottom right coordinate from image name
#     br_lon = float(img.split('_')[4] + '.' + img.split('_')[5]) #lon = x                                       
#     br_lat = float(img.split('_')[2] + '.' + img.split('_')[3]) #lat = y
    
#     #extract top left coordinate from image name
#     tl_lon = float(img.split('_')[8] + '.' + (img.split('_')[9]).split('.')[0]) #lon = x
#     tl_lat = float(img.split('_')[6] + '.' + img.split('_')[7]) #lat = y    
    
#     #FOR GREECE ONLY - extract bottom right coordinate from image name
    br_lon = float(img.split('_')[8] + '.' + img.split('_')[9]) #lon = x                                       
    br_lat = float(img.split('_')[6] + '.' + img.split('_')[7]) #lat = y
    
    #extract top left coordinate from image name
    tl_lon = float(img.split('_')[12] + '.' + (img.split('_')[13]).split('.')[0]) #lon = x
    tl_lat = float(img.split('_')[10] + '.' + img.split('_')[11]) #lat = y
    
    #image dimensions
    img_w = detections['img_width'].iloc[i]
    img_h = detections['img_height'].iloc[i]
    width = abs(br_lon - tl_lon) 
    height = abs(tl_lat - br_lat)
    
    #decimal degrees per pixel
    res_x = width / img_w #image width
    res_y = height / img_h #image height
    
    #res
    lat = tl_lat * math.pi / 180
    res = 156543.04 * math.cos(lat) / (2 ** 18)
    
    #convert bounding box to centroid
    xmin = detections['xmin'].iloc[i]
    ymin = detections['ymin'].iloc[i]
    xmax = detections['xmax'].iloc[i]
    ymax = detections['ymax'].iloc[i]
    x = (xmin + xmax) / 2
    y = (ymin + ymax) / 2
    
    #convert centroid point to lat/lon   
    x_center = tl_lon + (x * res_x) 
    y_center = tl_lat - (y * res_y) 
    
    #estimate radius of detection
    w = xmax-xmin 
    h = ymax-ymin 
    radius = (((w+h)/2)/4)*1.1
    diameter = radius * 2
    lon_list.append(x_center)
    lat_list.append(y_center)
    radius_list.append(radius)
    diameter_list.append(diameter)

In [41]:
#add attributes to df
d2 = selected.reset_index(drop=True) #super important
d2 = d2.drop(columns=['label'])
d2['x']=lon_list
d2['y']=lat_list
d2['radius']=np.round(radius_list,2)
d2['diameter']=np.round(diameter_list,2)
print(len(d2), 'total SIMRDWN predictions')

#add geometry information to df 
geometry = [Point(i) for i in zip(lon_list, lat_list)]
d2 = gpd.GeoDataFrame(d2, geometry=geometry, crs=wgs84)
d2 = d2.to_crs({'init': crs})
print(d2.crs)
d2.head()

17834 total SIMRDWN predictions
{'init': 'EPSG:2100'}


,img_file,img_width,img_height,xmin,ymin,xmax,ymax,x,y,radius,diameter,geometry
0,image_18_35_41838_24_96507_40_5061_22_66807_40...,2234,2175,1706,1195,1768,1335,22.665405,40.509811,27.78,55.55,POINT (386779.111440935 4484912.990089023)
1,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,1111,954,1133,978,20.857389,38.949200,6.32,12.65,POINT (227501.2774731213 4315549.554625049)
2,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,1145,954,1168,979,20.857574,38.949198,6.60,13.20,POINT (227517.3122310978 4315548.76941192)
3,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,1023,953,1048,982,20.856925,38.949194,7.43,14.85,POINT (227461.0303683727 4315550.248281476)
4,image_18_36_56134_26_39112_38_94454_20_86329_3...,2222,2083,1195,954,1217,978,20.857840,38.949200,6.32,12.65,POINT (227540.338078235 4315548.206635939)


In [42]:
#select predictions within search area
# search_area = gpd.read_file('../0_search_areas/4_search_area/search_area_100m_' + n + '.shp')
# search_area = search_area.to_crs({'init': crs})

# d2 = sjoin(d2, search_area, how='inner', op='within')
# print(len(d2), 'predictions within search area')

#select predictions by diameter
d = 55
d2 = d2[d2['diameter'] <= d]
geometry = [Point(i) for i in zip(d2['x'], d2['y'])]

d2 = d2[['radius', 'geometry']]
print(len(d2), 'total SIMRDWN predictions less than or equal to', d, 'meters')
d2.head()

15982 total SIMRDWN predictions less than or equal to 55 meters


,radius,geometry
1,6.32,POINT (227501.2774731213 4315549.554625049)
2,6.60,POINT (227517.3122310978 4315548.76941192)
3,7.43,POINT (227461.0303683727 4315550.248281476)
4,6.32,POINT (227540.338078235 4315548.206635939)
5,7.43,POINT (227471.9500476926 4315549.639538817)


In [43]:
#aggregate predictions 
buffer = gpd.GeoDataFrame(geometry = d2.buffer(10)) #buffer by 10 meters
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')
buffer_exploded = explode(buffer_dis)    
print(len(d2), 'predictions aggregated to', len(buffer_exploded), 'predictions')

15982 predictions aggregated to 5215 predictions


In [44]:
#calculate mean radius of aggregated predictions
d2_ag = gpd.sjoin(d2, buffer_exploded, how="inner", op='intersects')
centroids = gpd.GeoDataFrame(geometry = buffer_exploded.centroid, crs=wgs84)
centroids['radius']=d2_ag.groupby('index_right')['radius'].mean()
centroids.crs={'init' : crs}
print(len(centroids), 'predictions')
centroids.head()

5215 predictions


,geometry,radius
0,POINT (658913.6626176616 3860549.098161156),7.29
1,POINT (573866.5145819896 3865617.751289293),9.62
2,POINT (573386.5307144186 3865947.72739468),27.36
3,POINT (582194.878166277 3865955.324920327),24.34
4,POINT (574324.6597086105 3866010.842547479),14.44


In [45]:
#generate farmsite polygons
b = 100
buffer = gpd.GeoDataFrame(geometry = centroids.buffer(b)) #buffer by 100m
buffer['Dissolve'] = 0
buffer_dis = buffer.dissolve(by='Dissolve')
buffer_exploded = explode(buffer_dis)    
buffer_exploded.crs={'init' : crs}
buffer_exploded.shape

(786, 1)

In [46]:
#add number of predictions per farmsite to farmsite polygons
count=sjoin(centroids, buffer_exploded, how='inner', op='within')
buffer_exploded['count']=count.groupby('index_right')['index_right'].count()
buffer_exploded['count']=count.groupby('index_right')['index_right'].count()
buffer_exploded["farm_id"] = buffer_exploded.index + 1 #add farm ID

print(len(buffer_exploded), 'total farmsite predictions based on farmsites')
buffer_exploded.head()

786 total farmsite predictions based on farmsites


,geometry,count,farm_id
0,"POLYGON ((659013.6626176616 3860549.098161156,...",1,1
1,"POLYGON ((573966.5145819896 3865617.751289293,...",1,2
2,"POLYGON ((573486.5307144186 3865947.72739468, ...",1,3
3,"POLYGON ((696026.6756155745 3867026.933824575,...",1,4
4,"POLYGON ((694052.9056566937 3867967.623363503,...",1,5


In [47]:
#select farmsites by number of predictions
p = 4 #number of predictions per farmsite
farmsites = buffer_exploded[buffer_exploded['count'] >= p]
print(len(farmsites), 'farm site predictions containing', p, 'or more net pen predictions')
farmsites.head()

250 farm site predictions containing 4 or more net pen predictions


,geometry,count,farm_id
31,"POLYGON ((702107.7713218166 3903123.821657344,...",24,32
51,"POLYGON ((838895.9166277597 4018949.651446423,...",13,52
52,"POLYGON ((838562.7464948746 4018604.709674394,...",18,53
53,"POLYGON ((498172.738799314 3929800.849287874, ...",4,54
69,"POLYGON ((714222.1894808528 4052633.277855517,...",37,70


In [48]:
#select predictions by farmsites
select = farmsites
select['predictions'] = 0
select = select.dissolve(by='predictions')
mask = centroids.within(select.loc[0, 'geometry'])
farmsites=farmsites.drop(['predictions'], axis=1)
d2_clip = centroids.loc[mask]

print(len(d2_clip), 'net pen predictions within farm sites')
print(d2_clip.crs)
d2_clip.head()

C:\Users\Zack\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


4581 net pen predictions within farm sites
{'init': 'EPSG:2100'}


,geometry,radius
20,POINT (702146.2790860378 3903226.552240553),8.940
21,POINT (702105.460521807 3903237.317828557),8.665
22,POINT (702056.793952149 3903209.713557327),6.880
23,POINT (702081.1023536371 3903220.106714786),7.700
24,POINT (702197.4974791425 3903261.987968508),8.110


In [49]:
#add id and farm_id to predictions
d2_clip_id=sjoin(farmsites, d2_clip, how='right', op='intersects')
d2_clip_id=d2_clip_id.reset_index(drop=True)
d2_clip_id["id"] = d2_clip_id.index + 1 
d2_clip_id['diameter']=d2_clip_id['radius'] * 2
d2_clip=d2_clip_id[['id', 'farm_id', 'radius', 'diameter', 'geometry']]
print(len(d2_clip_id), 'net pen predictions')
d2_clip.head()

4581 net pen predictions


,id,farm_id,radius,diameter,geometry
0,1,32,6.880,13.76,POINT (702056.793952149 3903209.713557327)
1,2,32,7.700,15.40,POINT (702081.1023536371 3903220.106714786)
2,3,32,8.940,17.88,POINT (702146.2790860378 3903226.552240553)
3,4,32,8.665,17.33,POINT (702105.460521807 3903237.317828557)
4,5,32,8.250,16.50,POINT (702171.721853883 3903244.393861066)


In [50]:
# #generate farmsite extents
# envelope = gpd.GeoDataFrame(geometry = farmsites.envelope)
# farmsites2=farmsites.copy()
# farmsites2['geometry'] = envelope['geometry']
# print(len(farmsites2), 'farm site predictions')
# farmsites2.head()

In [51]:
# #exclusion of false positives
# path= f + n + '_exclude.kml'
# gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
# exclude = gpd.read_file(path, driver='KML')
# exclude = exclude.to_crs({'init': crs})

# #create exclusion mask
# exclude['Dissolve'] = 0
# exclude_dis = exclude.dissolve(by='Dissolve')

# #eliminate net pen false positives
# mask = ~d2_clip.within(exclude_dis.loc[0, 'geometry'])
# d2_clip = d2_clip.loc[mask]
# print('final number of predicted net pens:', len(d2_clip))

# #eliminate farmsite false positives
# mask2 = ~farmsites2.within(exclude_dis.loc[0, 'geometry'])
# farmsites2 = farmsites2.loc[mask2]
# print('final number of predicted farm sites:', len(farmsites2))

In [52]:
# #generate net pen buffers
# def buffer(row):
#      return row.geometry.buffer(row.radius)   
    
# d2_clip2 = d2_clip.copy()
# d2_clip2.crs

# buff = d2_clip2['geometry'] = d2_clip2.apply(buffer, axis=1)
# circles = gpd.GeoDataFrame(d2_clip2, geometry = buff, crs={'init': crs})

In [53]:
# print(len(circles), circles.crs)
# print(len(d2), d2.crs)
# print(len(d2_clip), d2_clip.crs)
# print(len(farmsites2), farmsites2.crs)

In [54]:
# #export buffered centroids
# circles.to_file(f + n + '_simrdwn_pens.shp')
# circles['geometry'] = circles['geometry'].to_crs(epsg=4326)
# circles.to_file(f + n + '_simrdwn_pens.geojson', driver='GeoJSON')

# #export all centroids
# d2 = d2.to_crs({'init': crs})
# d2.to_file(f + n + '_simrdwn.shp')
# d2['geometry'] = d2['geometry'].to_crs(epsg=4326)
# d2.to_file(f + n + '_simrdwn.geojson', driver='GeoJSON')

#export centroids within farmsites
d2_clip.to_file(f + n + '_simrdwn_pts.shp')
d2_clip['geometry'] = d2_clip['geometry'].to_crs(epsg=4326)
d2_clip.to_file(f + n + '_simrdwn_pts.geojson', driver='GeoJSON')

#export farmsites
# farmsites.to_file(f + n + '_simrdwn_farmsites.shp')
# farmsites['geometry'] = farmsites['geometry'].to_crs(epsg=4326)
# farmsites.to_file(f + n + '_simrdwn_farmsites.geojson', driver='GeoJSON')

# #export farmsites extents
# farmsites2.to_file(f + n + '_simrdwn_farmsites_ext.shp')
# farmsites2['geometry'] = farmsites2['geometry'].to_crs(epsg=4326)
# farmsites2.to_file(f + n + '_simrdwn_farmsites_ext.geojson', driver='GeoJSON')

C:\Users\Zack\Anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
